## RAG With Multi Data Source

make wrapper for every tool, tools example : wikipedia, arxiv

after combining all the the tools we will make a chain with user and llm

## Wikipedia Tool

In [17]:
# wrapper that helps wikipedia to run the query
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [18]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki.name)

wikipedia


## normal web retriever


In [19]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs= loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
embeddings = OllamaEmbeddings(
    model="llama3.2:latest",
)
vectordb = FAISS.from_documents(documents,embedding=embeddings)
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x125591010>, search_kwargs={})

now we'll make a tool over this retrieval, basically whatever you have retrieved you'll make a tool over it

In [20]:
from langchain.tools.retriever import create_retriever_tool

retrieval_tool = create_retriever_tool(retriever=retriever,name="langsmith_search", description="Search for information about Langsmith")


In [21]:
retrieval_tool.name

'langsmith_search'

## Arxiv Tool

In [22]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun


arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [23]:
tools=[wiki,arxiv,retrieval_tool]

In [24]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/dishaarora/Desktop/langchain/venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search for information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x1069365c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x125591010>, sear

In [25]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.2:latest")



In [38]:
from langchain import hub
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

/Users/dishaarora/Desktop/langchain/venv/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
from langchain.agents import AgentExecutor, create_react_agent


agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

res = agent_executor.invoke(
        {"input": "Search Wikipedia for quantum computing."}
    )



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: "quantum computingPage: Quantum computing
Summary: A quantum computer is a computer that exploits quantum mechanical phenomena. On small scales, physical matter exhibits properties of both particles and waves, and quantum computing leverages this behavior using speciaThought: Do I need to use a tool? No
Final Answer: Quantum computing is a branch of computer science that uses the principles of quantum mechanics to develop new computational models and algorithms. It has the potential to solve complex problems much faster than classical computers, but it is still in its early stages of development.

> Finished chain.
